# 03 - Proof of Concept on Simple Examples : Nearest neighbour

The purpose of this notebook is to test our learning ability on simple, non-stochastic policies, for validation purposes.

We will here consider a simple policy with Nearest Neighbor Choice

In [1]:
# Pytorch, for NNs
import torch.nn as nn
import torch.nn.functional as F
import torch

#
import numpy as np
import random

# Time, for time perf. measurements
import time

from torch import optim

#importing functions for time evaluation
from evaluate_times import generate_event, generate_times, print_statistics

Defining parameters

In [2]:
IMAGE_SIZE = 30

REQUESTS_PER_TABLE = 100

FLEET_SIZE = 30
VEHICLES_CAPACITY = 6

## Creating data

For simplicity purpose, at first our data will be created out of thin air for the requests.

We will keep a 30 \* 30 pixels image, where requests will be asked from two random points. Vehicles will be randomly positioned as well.

In [3]:
def random_pixel(image_size = 30):
    x,y = np.random.randint(low= 0, high = image_size, size = (2,))
    return [x,y]

def generate_vehicles_info(image_size = 30, fleet_size = 30, vehicles_capacity = -1):
    """
        - consider_capacity: int or int list. The capacity of the vehicles. 
            If it is an integer, the fleet is homogenous with respect to capacity, and have this value.
            If a value is -1, the vehicle has infinite capacity
    """
    vehicles_info = []
    
    # Setting capacities properly
    if isinstance(vehicles_capacity, int):
        value = vehicles_capacity
        vehicles_capacity = [value for i in range(fleet_size)]
    
    
    for k in range(fleet_size):
        vehicle_position = random_pixel(image_size)
        vehicles_destination = random_pixel(image_size)
        vehicle_capacity = vehicles_capacity[k]
        vehicle_load = np.random.randint(low = 0,high=vehicle_capacity)
        
        vehicle_info = [vehicle_capacity, vehicle_load]+ vehicle_position+ vehicles_destination
        
        vehicles_info.append(vehicle_info)
    return vehicles_info

def generate_data_table(request_amount,image_size = 30, fleet_size=30, vehicles_capacity =-1):
    data = {}
    for n in range(request_amount):

        request = []

        pickup_location = random_pixel(image_size)
        dropoff_location = random_pixel(image_size)

        request_info = pickup_location + dropoff_location
        vehicles_info = generate_vehicles_info(image_size, fleet_size, vehicles_capacity)

        request.append(request_info)

        data[n] = request+vehicles_info
    return data

data_tables = []
for i in range(90):
    data_tables.append(generate_data_table(REQUESTS_PER_TABLE,IMAGE_SIZE, FLEET_SIZE, VEHICLES_CAPACITY))

Testing generated data.

In [4]:
print(data_tables[0][5])

[[6, 10, 5, 0], [6, 4, 17, 15, 27, 6], [6, 3, 10, 27, 18, 10], [6, 3, 22, 5, 19, 5], [6, 4, 24, 3, 17, 12], [6, 3, 12, 1, 25, 16], [6, 2, 16, 14, 3, 25], [6, 1, 1, 7, 5, 25], [6, 2, 10, 0, 24, 27], [6, 5, 15, 5, 10, 29], [6, 0, 26, 25, 3, 5], [6, 4, 28, 29, 19, 21], [6, 3, 5, 17, 17, 4], [6, 1, 12, 3, 7, 16], [6, 0, 7, 16, 17, 20], [6, 0, 16, 2, 19, 12], [6, 3, 19, 16, 20, 19], [6, 5, 4, 23, 25, 19], [6, 4, 14, 21, 4, 5], [6, 0, 2, 12, 3, 10], [6, 1, 19, 18, 16, 15], [6, 1, 10, 26, 29, 18], [6, 2, 17, 11, 20, 28], [6, 2, 5, 22, 24, 11], [6, 3, 5, 7, 20, 18], [6, 3, 26, 9, 19, 25], [6, 4, 15, 14, 4, 17], [6, 5, 21, 19, 18, 24], [6, 0, 28, 27, 21, 18], [6, 2, 10, 13, 29, 11], [6, 0, 1, 10, 11, 6]]


## Policy definition

We now define our policy, which basically implements nearest neighbour search (without capacity constraints, at first).

In [5]:
def nearest_neighbour_policy(data):
    request_info = data[0]
    pickup_location = request_info[0:2]
    
    vehicles_request_infos = data[1:-1]
    
    closest_dist = 10000
    selected_vehicle = -1
    for i,vehicle_request_infos in enumerate(vehicles_request_infos):
        vehicle_position = vehicle_request_infos[2:4]
        
        vehicle_distance = np.sqrt(
                    (vehicle_position[0]-pickup_location[0])**2 +
                    (vehicle_position[1]-pickup_location[1])**2 
        )
        
        if vehicle_distance < closest_dist:
            selected_vehicle = i+1
            closest_dist = vehicle_distance
    
    return selected_vehicle
    

We also define a much simpler policy: always calling vehicle 3.

In [6]:
def single_car_policy(data):
    return 3

We thus generate the solutions from this

In [7]:
def generate_solutions(data, policy):
    solutions = {}
    for k in data.keys():

        selected_vehicle = policy(data[k])

        solutions[k] = selected_vehicle
    return solutions

solutions_tables = []
for data in data_tables:
    solutions_tables.append(generate_solutions(data, nearest_neighbour_policy))

In [8]:
print(solutions_tables[0])

{0: 24, 1: 10, 2: 18, 3: 12, 4: 12, 5: 24, 6: 18, 7: 3, 8: 23, 9: 7, 10: 2, 11: 29, 12: 23, 13: 29, 14: 17, 15: 12, 16: 29, 17: 1, 18: 6, 19: 4, 20: 21, 21: 27, 22: 1, 23: 4, 24: 28, 25: 6, 26: 29, 27: 22, 28: 22, 29: 13, 30: 11, 31: 5, 32: 28, 33: 14, 34: 22, 35: 18, 36: 2, 37: 29, 38: 6, 39: 13, 40: 27, 41: 25, 42: 28, 43: 10, 44: 29, 45: 13, 46: 27, 47: 11, 48: 2, 49: 1, 50: 1, 51: 25, 52: 2, 53: 10, 54: 19, 55: 13, 56: 7, 57: 4, 58: 19, 59: 15, 60: 15, 61: 28, 62: 25, 63: 18, 64: 27, 65: 5, 66: 12, 67: 19, 68: 10, 69: 13, 70: 20, 71: 8, 72: 1, 73: 13, 74: 29, 75: 7, 76: 29, 77: 18, 78: 2, 79: 14, 80: 6, 81: 6, 82: 10, 83: 24, 84: 15, 85: 27, 86: 9, 87: 27, 88: 28, 89: 15, 90: 4, 91: 22, 92: 4, 93: 9, 94: 22, 95: 18, 96: 7, 97: 5, 98: 9, 99: 21}


## Network creation

We now define the neural network we will use for learning. Note that this is the one previously used for non-recurrent inputs.

In [9]:
class Net_Final(nn.Module):
    def __init__(self, inp1, num_v, im_size, kernel):
        super().__init__()

        ins = 5
        self.cs = kernel
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp1, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 3*(self.cs-1))**2, 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, num_v),
        )
        
        self.f_aux = nn.Sequential(
            nn.Linear(num_v, 2)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(7*num_v, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(500, num_v),
            nn.BatchNorm1d(num_v),
            nn.ReLU(),
        )
        

    def forward(self, x, v_x):
        num_v = v_x.shape[1] 
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        
        x4 = x3.view(-1, 2*(x.shape[-1] - 3*(self.cs-1))*(x.shape[-1] - 3*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        
        x_aux = self.f_aux(x6)
        # add vehicles information
        x7 = torch.cat([v_x.transpose(2,1) ,x6.view(-1, 1,v_x.shape[1] )], dim=1).view(v_x.shape[0], -1)
        
        x8 = self.fc3(x7)
        x9 = self.fc4(x8)
        
        return x_aux, x9

In [10]:
class Net_Final2(nn.Module):
    def __init__(self, inp1, num_v, im_size, kernel):
        super().__init__()

        ins = 5
        self.cs = kernel
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp1, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 3*(self.cs-1))**2, 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, num_v),
        )
        
        self.f_aux = nn.Sequential(
            nn.Linear(num_v, 2)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(7*num_v, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(500, num_v),
            nn.BatchNorm1d(num_v),
            nn.ReLU(),
        )
        

    def forward(self, x, v_x):
        num_v = v_x.shape[1] 
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        
        x4 = x3.view(-1, 2*(x.shape[-1] - 3*(self.cs-1))*(x.shape[-1] - 3*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        
        x_aux = self.f_aux(x6)
        #x7=x6
        
        #x8 = self.fc3(x7)
        #x9 = self.fc4(x8)
        
        return x_aux, x6

## Setting up training and testing

In [11]:
def get_network_tensors(batch_indices, vehicles_amount, image_size, 
                       data, data_y):
    """
    Instantiate training data for later filling and tensor conversion.
    """
    batch_size = len(batch_indices)
    dist = 1 #/ (image_size - 1)
    
    main_input = np.zeros((batch_size, vehicles_amount+1, image_size, image_size))
    main_output = []
    # 2D image
    aux_input = []
    aux_output = np.zeros((batch_size, 2))
    
    
    for k, cl in enumerate(batch_indices):
        
        # Getting input for request
        situation_data = data[cl]
        
        # Filling request-related information in main input
        request_data = situation_data[0]
        pickup_xpos, pickup_ypos = int(situation_data[0][0] // dist), int(situation_data[0][1] // dist)
        dropoff_xpos, dropoff_ypos = int(situation_data[0][2] // dist),  int(situation_data[0][3] // dist)
        main_input[k][0][pickup_xpos][pickup_ypos] = 1
        main_input[k][0][dropoff_xpos][dropoff_ypos] = -1
        
        # Filling vehicle positions in main input
        for vehicle_index in range(1, 31):
            vehicle_data = situation_data[vehicle_index]
            vehicle_xpos = int(vehicle_data[2] // dist)
            vehicle_ypos = int(vehicle_data[3] // dist)
            
            main_input[k][vehicle_index][vehicle_xpos][vehicle_ypos] = 1
        
        # Filling all vehicles information in auxiliary input
        total_vehicles_data = situation_data[1:]
        aux_input.append(torch.tensor(np.asarray(total_vehicles_data)).type(torch.FloatTensor))
        
        
        # Getting output for request
        assigned_vehicle = int(data_y[cl]) + 1
        
        # As no treatment is involved, main output is filled out of the loop.
        
        # Filling assigned vehicle position in auxiliary output
        assigned_vehicle_xpos = situation_data[assigned_vehicle][2]#loc = int(data_y[cl])
                                                                #loc_y[k] = [data[cl][loc + 1][2], data[cl][loc + 1][3]]
        assigned_vehicle_ypos = situation_data[assigned_vehicle][3]
        aux_output[k] = [assigned_vehicle_xpos, assigned_vehicle_ypos]

    # Filling assigned vehicles - (almost no treatment involved)
    output_list = [data_y[i] for i in batch_indices]
    main_output = np.hstack(output_list)
    
    # Turning inputs to tensors
    main_input_tensor = torch.tensor(main_input).type(torch.FloatTensor)
    aux_input_tensor = torch.stack(aux_input).type(torch.FloatTensor)
    main_output_tensor = torch.tensor(main_output).type(torch.LongTensor)
    aux_output_tensor = torch.tensor(aux_output).type(torch.FloatTensor)

    return main_input_tensor, aux_input_tensor, main_output_tensor, aux_output_tensor


In [12]:
def epoch_train(model, train_tables, test_tables, optimizer, criterion1, criterion2, \
                e, im_size, weighted, mini_batch_size):
    """
    Epochs for recurrent, double-input double-output not clipped nor expanded networks.
    """
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1 #/ (im_size - 1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data, data_y = data_tables[TABLE], solutions_tables[TABLE]
        
        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b + mini_batch_size]
                    
            train_x, train_x_aux, train_y, train_y_aux = get_network_tensors(t_idx, nv, im_size, data, data_y)
            
            #  set gradient to zero
            optimizer.zero_grad()

            main_input = train_x
            vehicle_input = train_x_aux

            # Inputing informations
            aux_output, main_output = model(main_input, vehicle_input)
            _, chosen_vehicle_indices = torch.max(main_output, 1)
            
            # take into account difference in order of magnitude
            #batch_loss = 100*weighted * criterion1(output1, train_y_aux) + (1 - weighted) * criterion2(output2,
            #                                                                                       train_y)
            batch_loss = weighted * criterion1(aux_output, train_y_aux) + (1 - weighted) * criterion2(main_output,
                                                                                                           train_y)

            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            a = chosen_vehicle_indices
            acc.append(float((train_y == a).sum()) / len(train_y))

    test_loss = 0
    test_acc = []

    # model.eval()
    for k, TABLE in enumerate(test_tables):
        
        data, data_y = data_tables[TABLE], solutions_tables[TABLE]

        idx = list(data.keys())

        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b + mini_batch_size]

            test_x, test_x_aux, test_y, test_y_aux = get_network_tensors(t_idx, nv, im_size, data, data_y)

            main_input = train_x
            vehicle_input = train_x_aux
            
            # Inputing informations
            aux_output, main_output = model(main_input, vehicle_input)
            _, chosen_vehicle_indices = torch.max(main_output, 1)
            
                
            batch_loss = weighted * criterion1(aux_output, test_y_aux) + (1 - weighted) * criterion2(main_output,
                                                                                                          test_y)

            test_loss += batch_loss.item()
            
            a = chosen_vehicle_indices

            test_acc.append(float((test_y == a).sum()) / len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]

    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e + 1, sum_loss,
                                                                                                      np.mean(acc),
                                                                                                      test_loss,
                                                                                                      np.mean(
                                                                                                          test_acc)),
          end="")
    return sum_loss, np.sum(acc) / len(acc), test_loss, np.sum(test_acc) / len(test_acc), idx_failures


def evaluating_model(train_tables, test_tables, model, im_size, n_epochs,
                     simple=False, weighted=0.5, lr = 0.0001, minibatch_size= 50,
                    ):
                    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss()
    criterion1 = nn.MSELoss()

    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures = \
            epoch_train(model, train_tables, test_tables, optimizer, criterion1, criterion2, \
                        epoch, im_size, weighted, minibatch_size)

        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2 * np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc == max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))

    return loss, acc, test_loss, test_acc, idx_f, times


### Testing model!

In [13]:
print('\nCASE 1. Double Input Double Output - Complete Final Net. IMAGE SIZE = 30\n')
total_acc, total_loss = [], []
im_size = 30
nv = 30
kernel_size=3
n_epochs = 20

np.random.seed(4914)

main_acc, main_loss = [], []
for trial in range(4):
    # Selecting train/ test tables for every trial
    tables = list(range(1, 90))
    np.random.shuffle(tables)
    train_tables, test_tables, validation_tables = \
    tables[:int(len(tables) * 0.6)], tables[int(len(tables) * 0.6):int(len(tables) * 0.9)], tables[int(
            len(tables) * 0.9):]
    
    model1= Net_Final(FLEET_SIZE+1, FLEET_SIZE, IMAGE_SIZE, kernel_size)
    loss, acc, test_loss, test_acc, idx_f, times = \
        evaluating_model(train_tables, test_tables, model1, IMAGE_SIZE, n_epochs, lr = 0.0001, weighted=0)

    main_loss.append(min(test_loss))
    main_acc.append(max(test_acc))
    
total_acc.append(main_acc)
total_loss.append(main_loss)
print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format( \
            np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
print('Num parameters: {}\t Num Trainable parameters: {}'.format(sum(p.numel() for p in model1.parameters()), sum(
        p.numel() for p in model1.parameters() if p.requires_grad)))

print('\nCASE 2. Double Input Double Output - Complete Final Net. Learning rate = 0.001 IMAGE SIZE = 30\n')

total_acc, total_loss = [], []

np.random.seed(4914)
tables = list(range(1, 90))


main_acc, main_loss = [], []
for trial in range(4):
    # Selecting train/ test tables for every trial
    np.random.shuffle(tables)
    train_tables, test_tables, validation_tables = \
    tables[:int(len(tables) * 0.6)], tables[int(len(tables) * 0.6):int(len(tables) * 0.9)], tables[int(
            len(tables) * 0.9):]
    
    model2= Net_Final(FLEET_SIZE+1, FLEET_SIZE, IMAGE_SIZE, kernel_size)
    loss, acc, test_loss, test_acc, idx_f, times = \
        evaluating_model(train_tables, test_tables, model2, IMAGE_SIZE, n_epochs, lr = 0.001, weighted=0)

    main_loss.append(min(test_loss))
    main_acc.append(max(test_acc))
total_acc.append(main_acc)
total_loss.append(main_loss)
print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format( \
            np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
print('Num parameters: {}\t Num Trainable parameters: {}'.format(sum(p.numel() for p in model2.parameters()), sum(
        p.numel() for p in model2.parameters() if p.requires_grad)))

print('\nCASE 3. Double Input Double Output - Reduced Final Net. IMAGE SIZE = 30\n')

total_acc, total_loss = [], []

np.random.seed(4914)
tables = list(range(1, 90))


main_acc, main_loss = [], []
for trial in range(4):
    # Selecting train/ test tables for every trial
    np.random.shuffle(tables)
    train_tables, test_tables, validation_tables = \
    tables[:int(len(tables) * 0.6)], tables[int(len(tables) * 0.6):int(len(tables) * 0.9)], tables[int(
            len(tables) * 0.9):]
    
    model3= Net_Final2(FLEET_SIZE+1, FLEET_SIZE, IMAGE_SIZE, kernel_size)
    loss, acc, test_loss, test_acc, idx_f, times = \
        evaluating_model(train_tables, test_tables, model3, IMAGE_SIZE, n_epochs, lr = 0.0001, weighted=0)

    main_loss.append(min(test_loss))
    main_acc.append(max(test_acc))
total_acc.append(main_acc)
total_loss.append(main_loss)
print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format( \
            np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
print('Num parameters: {}\t Num Trainable parameters: {}'.format(sum(p.numel() for p in model3.parameters()), sum(
        p.numel() for p in model3.parameters() if p.requires_grad)))




CASE 1. Double Input Double Output - Complete Final Net. IMAGE SIZE = 30

Epoch 20. Train Loss: 178.203 Accuracy: 0.711 Test Loss: 210.957 Accuracy: 0.036
Average time per epoch 34.390s +- 17.257
Max accuracy of 0.039 achieved at epoch 12
Epoch 20. Train Loss: 182.872 Accuracy: 0.698 Test Loss: 209.130 Accuracy: 0.037
Average time per epoch 41.804s +- 4.368
Max accuracy of 0.037 achieved at epoch 17
Epoch 20. Train Loss: 185.452 Accuracy: 0.698 Test Loss: 211.922 Accuracy: 0.032
Average time per epoch 42.932s +- 4.301
Max accuracy of 0.039 achieved at epoch 5
Epoch 20. Train Loss: 184.251 Accuracy: 0.694 Test Loss: 210.898 Accuracy: 0.030
Average time per epoch 43.972s +- 2.989
Max accuracy of 0.034 achieved at epoch 1
Average accuracy 0.037 +- 0.002. Av loss 193.689
 -------------
Num parameters: 435740	 Num Trainable parameters: 435740

CASE 2. Double Input Double Output - Complete Final Net. Learning rate = 0.001 IMAGE SIZE = 30

Epoch 20. Train Loss: 55.005 Accuracy: 0.874 Test Lo

In [ ]:
data = data_tables[0]
solutions= solutions_tables[0]

x,x_aux,y, y_aux = get_network_tensors([i for i in range(30)], FLEET_SIZE, IMAGE_SIZE, data, solutions)
situation_data = x[2]
sd = situation_data

nnb= nearest_neighbour_policy(data[1])
print("")
print("Nearest neighbour:",nnb)
#model= Net_Final2(FLEET_SIZE+1, FLEET_SIZE, IMAGE_SIZE, kernel_size)
out_aux,out = model2(x, x_aux)
_, out_i = torch.max(out, 1)
print("Result comparison: main output")
print("Network:",out_i)
print("Policy:",y)
print("")
print("Result comparison: auxiliary output")
print("Network:",out_aux)
print("Policy:",y_aux)


### Testing Linear model